# Predict impact of air quality on mortality rates

## Summary

This document is related to the kaggle inclass competition [Predict impact of air quality on mortality rates](https://inclass.kaggle.com/c/predict-impact-of-air-quality-on-death-rates)

We are going to to try to predict the death ratio from some enviromental features using keras library.

[NO UTILIZAR / VER VERSION 2]

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


seed = 7
np.random.seed(seed)

Using TensorFlow backend.


In [2]:
training = pd.read_csv('input/train.csv')
testing = pd.read_csv('input/test.csv')

training_original = training
testing_original = testing

In [3]:
training.head()

,Id,region,date,mortality_rate,O3,PM10,PM25,NO2,T2M
0,1,E12000001,2007-01-02,2.264,42.358,9.021,NaN,NaN,278.138
1,2,E12000001,2007-01-03,2.030,49.506,5.256,NaN,NaN,281.745
2,3,E12000001,2007-01-04,1.874,51.101,4.946,NaN,NaN,280.523
3,4,E12000001,2007-01-05,2.069,47.478,6.823,NaN,NaN,280.421
4,5,E12000001,2007-01-06,1.913,45.226,7.532,NaN,NaN,278.961


In [4]:
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18403 entries, 0 to 18402
Data columns (total 9 columns):
Id                18403 non-null int64
region            18403 non-null object
date              18403 non-null object
mortality_rate    18403 non-null float64
O3                18394 non-null float64
PM10              18394 non-null float64
PM25              15127 non-null float64
NO2               11833 non-null float64
T2M               18403 non-null float64
dtypes: float64(6), int64(1), object(2)
memory usage: 1.3+ MB


In [5]:
training.describe()

,Id,mortality_rate,O3,PM10,PM25,NO2,T2M
count,18403.000000,18403.000000,18394.000000,18394.000000,15127.000000,11833.000000,18403.000000
mean,9202.000000,1.301737,45.325857,13.712272,7.498714,12.045813,283.002235
std,5312.632838,0.304161,16.221330,7.421616,5.758357,8.296675,5.182186
min,1.000000,0.439000,0.988000,2.020000,0.904000,1.104000,265.562000
25%,4601.500000,1.102000,35.074250,8.656250,3.624000,6.056000,279.321500
50%,9202.000000,1.281000,45.836000,11.704500,5.636000,9.769000,283.270000
75%,13802.500000,1.474000,55.881000,16.589000,9.326500,15.858000,287.240500
max,18403.000000,2.841000,105.693000,60.627000,45.846000,76.765000,297.209000


In [6]:
tmp = pd.to_datetime(training['date'])
training_complete = training.loc[tmp.dt.year >= 2009]

In [7]:
def preprocess(df):
    
    df_columns = list(df.columns)
    
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month
    dummy_fields = ['region', 'month']
    for each in dummy_fields:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)

    fields_to_drop = ['date', 'region', 'Id', 'month']
    df = df.drop(fields_to_drop, axis=1)
    
    # scale numerical features


    quant_features = ['O3', 'PM10', 'PM25', 'NO2', 'T2M', 'mortality_rate']
    scaled_features = {}
    
    for each in quant_features:
        if each in df_columns:
            mean, std = df[each].mean(), df[each].std()
            scaled_features[each] = [mean, std]
            df.loc[:, each] = (df[each] - mean)/std

    return df
    

In [8]:
training_complete = preprocess(training_complete)
testing = preprocess(testing)

/Users/imac1/anaconda/envs/KERAS/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/imac1/anaconda/envs/KERAS/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
training_complete.head()

,mortality_rate,O3,PM10,PM25,NO2,T2M,region_E12000001,region_E12000002,region_E12000003,region_E12000004,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
6570,3.281967,-1.463134,-0.576597,-0.628081,-0.501865,-1.756626,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6571,4.084424,-1.256151,-0.717872,-0.692081,-0.088447,-1.647052,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6572,3.147651,-0.673306,0.065449,-0.831036,-0.678201,-1.903215,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6573,4.487375,-0.897132,0.341689,-0.329996,-0.855742,-1.510775,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6574,4.886881,-0.888806,-0.380870,-0.549994,-0.455943,-1.775410,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
training_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11833 entries, 6570 to 18402
Data columns (total 27 columns):
mortality_rate      11833 non-null float64
O3                  11833 non-null float64
PM10                11833 non-null float64
PM25                11833 non-null float64
NO2                 11833 non-null float64
T2M                 11833 non-null float64
region_E12000001    11833 non-null uint8
region_E12000002    11833 non-null uint8
region_E12000003    11833 non-null uint8
region_E12000004    11833 non-null uint8
region_E12000005    11833 non-null uint8
region_E12000006    11833 non-null uint8
region_E12000007    11833 non-null uint8
region_E12000008    11833 non-null uint8
region_E12000009    11833 non-null uint8
month_1             11833 non-null uint8
month_2             11833 non-null uint8
month_3             11833 non-null uint8
month_4             11833 non-null uint8
month_5             11833 non-null uint8
month_6             11833 non-null uint8
month_7        

In [13]:
X = training_complete.ix[:, training_complete.columns != 'mortality_rate']
y = training_complete.loc[:,'mortality_rate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [14]:
X_train.head()

,O3,PM10,PM25,NO2,T2M,region_E12000001,region_E12000002,region_E12000003,region_E12000004,region_E12000005,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
13323,0.972442,-0.267987,0.046002,-0.880933,1.104083,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6628,0.424884,-0.562195,-0.508777,-0.773179,-0.565493,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11727,-1.082789,3.458237,2.437293,1.671656,-1.314278,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9435,-0.864214,-0.699081,-0.623733,-0.401584,-0.399198,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
10294,0.436988,0.412738,0.443565,0.039556,-0.808765,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [43]:
# create model
model = Sequential()
model.add(Dense(50, input_dim=X_train.shape[1], kernel_initializer='uniform', activation='linear'))
model.add(Dense(1, activation='linear'))

In [44]:
# Compile model
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [45]:
# Fit the model
model.fit(X_train.values, y_train.values, epochs=1000, batch_size=100, verbose=0)

In [46]:
# evaluate the model on training data
scores = model.evaluate(X_train.values, y_train.values)
print("%s: %.2f" % (' RMSE', scores[1]**0.5))

6016/8283 [====================>.........] - ETA: 0s RMSE: 0.64


In [47]:
# evaluate the model on training data
scores = model.evaluate(X_test.values, y_test.values)
print("%s: %.2f" % (' RMSE', scores[1]**0.5))

1856/3550 [==============>...............] - ETA: 0s RMSE: 0.64


In [48]:
k = model.predict(X_test.values)
k[0:10] 

array([[-0.34064454],
       [-0.08276083],
       [ 0.1892373 ],
       [-0.20406395],
       [-0.39810318],
       [ 0.1166869 ],
       [ 1.41603947],
       [ 0.46950501],
       [ 0.54200214],
       [ 1.19032967]], dtype=float32)

In [49]:
y_test.head(10)

14748   -0.224046
7470     0.340774
10685   -0.079397
17336   -0.647660
15288   -0.754425
8475     0.871153
13516    1.615061
13227    2.028344
16759    0.292558
13132    1.553069
Name: mortality_rate, dtype: float64

Let's try to predict for the unknown cases:

In [22]:
testing.head()

,O3,PM10,PM25,NO2,T2M,region_E12000001,region_E12000002,region_E12000003,region_E12000004,region_E12000005,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1.644330,0.826342,1.611542,-0.387421,1.398670,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1.464307,0.928680,1.405083,-0.533894,0.960107,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0.610563,0.963271,1.390325,-0.255753,1.228402,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,-0.219870,0.000461,0.409607,-0.320852,0.835155,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0.166721,-0.435649,0.123609,-0.513438,0.799508,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Prediction and output file creation

In [23]:
testPredict = model.predict(testing.values)

In [24]:
#mean, std = scaled_features['mortality_rate']
mean, std = training_original['mortality_rate'].mean(),training_original['mortality_rate'].std()
predictions = testPredict*std + mean

In [25]:
output = testing_original[['Id']].copy()
output['mortality_rate'] = predictions

output.to_csv('output/keras_submission_1.csv', index = False)

In [26]:
output.head(10)

,Id,mortality_rate
0,18404,1.185962
1,18405,1.172253
2,18406,1.169857
3,18407,1.138178
4,18408,1.085692
5,18409,1.140116
6,18410,1.159549
7,18411,1.091857
8,18412,1.079288
9,18413,1.095026
